In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import cv2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import inspect
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import torch

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
pd.read_csv('/kaggle/input/shopee-product-matching/sample_submission.csv')
df=pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')

In [ ]:
df.head()

In [ ]:
x=LabelEncoder()
df['label_group']=x.fit_transform(df['label_group'].to_numpy())

In [ ]:
df['label_group'].value_counts()

In [ ]:
df.shape

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util

In [ ]:
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

In [ ]:
df['title'].head()

In [ ]:
#Comment after encoding and saving
title_embeddings = embedder.encode(df['title'])
for each in range(len(title_embeddings)):
    df.at[each,'title']=title_embeddings[each]
#comment this after saving embeddings and use the text file  by loading 
#torch.save(title_embeddings,'embed.txt')
del title_embeddings
del embedder

In [ ]:
# df['title']=torch.load('kaggle/output/kaggle/working/embed.txt')

In [ ]:
df.head()
directory='/kaggle/input/shopee-product-matching/train_images/'


In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import BatchNormalization,Input,LSTM,Dense,Input,Dropout,Concatenate,Flatten,Conv2D,MaxPooling2D,Flatten,Reshape,UpSampling2D
from keras.layers.merge import concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator

_datagen = ImageDataGenerator(
            rescale=1./255.,
            featurewise_center=False,
            featurewise_std_normalization=False,
            rotation_range=5,
            width_shift_range=0,
            height_shift_range=0,
            horizontal_flip=False,
            vertical_flip=False
            )
ec=Sequential()
ec.add(Input(shape=(160,160,1)))
ec.add(Conv2D(64,3,padding='same'))
ec.add(BatchNormalization())
ec.add(MaxPooling2D((2,2),strides=2,padding='same'))
ec.add(Conv2D(32,3,padding='same'))
ec.add(MaxPooling2D((2,2),strides=2,padding='same'))
ec.add(Conv2D(16,3,padding='same'))
ec.add(MaxPooling2D((2,2),strides=2,padding='same'))
ec.add(Conv2D(8,3,padding='same'))
ec.add(MaxPooling2D((2,2),strides=2,padding='same'))
ec.add(Flatten())
ec.add(Dense(100))
ec.add(BatchNormalization())
ec.add(Dense(800))
ec.add(BatchNormalization())
ec.add(Reshape((10,10,8)))
ec.add(Conv2D(8,3,padding='same'))
ec.add(UpSampling2D((2,2)))
ec.add(Conv2D(16,3,padding='same'))
ec.add(UpSampling2D((2,2)))
ec.add(Conv2D(32,3,padding='same'))
ec.add(UpSampling2D((2,2)))
ec.add(Conv2D(64,3,padding='same'))
ec.add(UpSampling2D((2,2)))
ec.add(Conv2D(1,3,padding='same'))
ec.summary()
ec.compile(loss='mse',optimizer='adam')
ec.fit_generator(_datagen.flow_from_dataframe(df, target_size=((160,160)),
                                                      color_mode='grayscale',
                                                      directory='/kaggle/input/shopee-product-matching/train_images/',
                                                      x_col='image',
                                                      class_mode='input',
                                                      shuffle=True,
                                                      batch_size=32,
                                                      seed=7,
                                                      subset='training'),epochs=2)
         

In [ ]:
bb=np.reshape(cv2.resize(cv2.imread(directory+df.loc[0,'image'],0),(160,160)),(160,160))
print(bb.shape)
print(ec.inputs)

In [ ]:
import tqdm
#Doing this to add array values to cell of df
df['image_embedding']=1
df=df.astype('object')
extractor = Model(inputs=ec.inputs,
                        outputs= ec.layers[10].output)

for i in tqdm.tqdm(range(len(df['image'].to_list()))):
    aux=np.array(extractor.predict(np.reshape(cv2.resize(cv2.imread(directory+df.loc[i,'image'],0),(160,160)),(1,160,160)))[0])
    df.at[i,"image_embedding"]=aux
    del aux



In [ ]:
df.head()

In [ ]:
matching_list=pd.merge(df,df,on='label_group')
matching_list=matching_list.drop(matching_list[matching_list.loc[:,'posting_id_x']==matching_list.loc[:,'posting_id_y']].index)
matching_list.reset_index()
matching_list=matching_list.drop('label_group',axis=1)
matching_list['similarity']=1
matching_list.shape

In [ ]:
non_matching_list=pd.DataFrame(columns=df.columns)
seen=[]
sample=3
for i in tqdm.tqdm(range(df.shape[0])):
    if(df.loc[i,'label_group'] not in seen):
        seen.append(df.loc[i,'label_group'])
        new_df_group=df[df['label_group']==int(df.loc[i,'label_group'])]
        new_df_nogroup=df.drop(df[df['label_group']==int(df.loc[i,'label_group'])].index)
        new_df_group['kw']=1
        new_df_nogroup['kw']=1
        if(i==0):
            non_matching_list=pd.merge(new_df_group.iloc[random.sample(range(0, new_df_group.shape[0]), min(sample,new_df_group.shape[0]))],new_df_nogroup.iloc[random.sample(range(0, new_df_nogroup.shape[0]), sample)],on='kw',suffixes=['_x','_y'])
        else:
            non_matching_list=non_matching_list.append(pd.merge(new_df_group.iloc[random.sample(range(0, new_df_group.shape[0]),  min(sample,new_df_group.shape[0]))],new_df_nogroup.iloc[random.sample(range(0, new_df_nogroup.shape[0]), sample)],on='kw',suffixes=['_x','_y']))
        del new_df_group
        del new_df_nogroup

In [ ]:
non_matching_list=non_matching_list.drop(columns=['label_group_x','label_group_y','kw'])
non_matching_list['similarity']=0

In [ ]:
non_matching_list.head()

In [ ]:
# non_matching_list['title_x']=1
# non_matching_list['image_embedding_x']=1
# non_matching_list['title_y']=1
# non_matching_list['image_embedding_y']=1
# non_matching_list=non_matching_list.astype('object')

# for i in tqdm.tqdm(range(non_matching_list.shape[0])):
#     ind=df['posting_id'][df['posting_id']==non_matching_list.iloc[i,non_matching_list.columns.get_loc("posting_id_x")]].index[0]
#     non_matching_list.at[i,'title_x']=[df.iloc[ind,df.columns.get_loc('title')]]
#     non_matching_list.at[i,'image_embedding_x']=[df.iloc[ind,df.columns.get_loc('image_embedding')]]
#     ind=df['posting_id'][df['posting_id']==non_matching_list.iloc[i,non_matching_list.columns.get_loc("posting_id_y")]].index[0]
#     non_matching_list.at[i,'title_y']=[df.iloc[ind,df.columns.get_loc('title')]]
#     non_matching_list.at[i,'image_embedding_y']=[df.iloc[ind,df.columns.get_loc('image_embedding')]]


In [ ]:
matching_list.head()

In [ ]:
data=matching_list.append(non_matching_list)

In [ ]:
del matching_list
del non_matching_list


In [ ]:
data['similarity'].value_counts()

In [ ]:
print(data.shape)
data=data.reset_index()

In [ ]:
data=data.drop(columns=['index'])

In [ ]:
data.head()

In [ ]:
X_tr1,X_tr2,X_tr3,X_tr4=(data["title_x"].to_numpy(),data["title_y"].to_numpy(),data["image_embedding_x"].to_numpy(),data["image_embedding_y"].to_numpy())

In [ ]:
X_tr1=np.array(list(map( lambda vl: list(vl),X_tr1)))
X_tr2=np.array(list(map( lambda vl: list(vl),X_tr2)))
X_tr3=np.array(list(map( lambda vl: list(vl),X_tr3)))
X_tr4=np.array(list(map( lambda vl: list(vl),X_tr4)))


In [ ]:
#print(X_tr3)
print(X_tr1.shape)

# ddd=np.array([np.array([2,4,5,3]),np.array([4,2,1,2])])
# print(ddd.shape)

In [ ]:
similarity=np.array(data["similarity"])

In [ ]:
data['similarity'].value_counts()

In [ ]:
np.argwhere(np.isnan(X_tr3))


In [ ]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout,Concatenate,Flatten,Conv2D
from keras.layers.merge import concatenate

model0=Sequential()
model0.add(Dense(300,activation='relu',input_shape=(768,)))
model0.add(Dense(100,activation='relu'))
model0.add(Dense(30,activation='relu'))
model0.summary()

model1=Sequential()
model1.add(Dense(300,activation='relu',input_shape=(768,)))
model1.add(Dense(100,activation='relu'))
model1.add(Dense(30,activation='relu'))
model1.summary()



model2=Sequential()
model2.add(Dense(100,activation='relu',input_shape=(100,)))
model2.add(Dense(50,activation='relu'))
model2.summary()


model3=Sequential()
model3.add(Dense(100,activation='relu',input_shape=(100,)))
model3.add(Dense(50,activation='relu'))
model3.summary()


modelx=concatenate([model0.output,model1.output,model2.output,model3.output])
modelx=Flatten()(modelx)
modelx=Dense(50,activation='relu')(modelx)
modelx=Dense(1,activation='sigmoid')(modelx)

model=Model(inputs=[model0.input,model1.input,model2.input,model3.input],outputs=modelx)
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam')
model.fit([X_tr1,X_tr2,X_tr3,X_tr4],similarity,epochs=2,batch_size=5)

In [ ]:
df_test=pd.read_csv('/kaggle/input/shopee-product-matching/test.csv')

In [ ]:
embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
title_embeddings = embedder.encode(df_test['title'])
for each in range(len(title_embeddings)):
    df_test.at[each,'title']=title_embeddings[each]
#comment this after saving embeddings and use the text file  by loading 
#torch.save(title_embeddings,'embed.txt')
del title_embeddings
del embedder

In [ ]:
directory='/kaggle/input/shopee-product-matching/test_images/'
df_test['image_embedding']=1
df_test=df_test.astype('object')
for i in tqdm.tqdm(range(len(df_test['image'].to_list()))):
    aux=np.array(extractor.predict(np.reshape(cv2.resize(cv2.imread(directory+df_test.loc[i,'image'],0),(160,160)),(1,160,160)))[0])
    df_test.at[i,"image_embedding"]=aux

In [ ]:
# df.head()

In [ ]:
df_test.head()

In [ ]:
df_test['title'][0][0]

In [ ]:
all_post=[]
for i,tes in tqdm.tqdm(df_test.iterrows()):
    all_comp=[]
    X_ts1,X_ts2=(tes["title"],tes["image_embedding"])
    X_ts1,X_ts2= np.reshape(X_ts1,(1,X_ts1.shape[0])),np.reshape(X_ts2,(1,X_ts2.shape[0]))
    for j,tr  in df_test.iterrows():
        X_ts3,X_ts4=(tr["title"],tr["image_embedding"])
        X_ts3,X_ts4=np.reshape(X_ts3,(1,X_ts3.shape[0])),np.reshape(X_ts4,(1,X_ts4.shape[0]))
        compare_val=model.predict([X_ts1,X_ts3,X_ts2,X_ts4])
        all_comp.append({"posting_id":tr['posting_id'], "val":compare_val})
    all_post.append({"posting_id":tes['posting_id'],"vals":all_comp})
    

In [ ]:
print(all_post)

In [ ]:
final=[]
for each in all_post:
    lst=each['vals']
    topk=sorted(lst, key=lambda i:i['val'],reverse=True)[: len(lst) if 50>=len(lst) else 50 ]
    acul=[]
    for val in topk:
        if(float(val['val'])>0.5):
            acul.append(val['posting_id'])  
    final.append(acul)

In [ ]:
string_list=[]
for each in final:
    string_list.append(' '.join(each))

In [ ]:
string_list

In [ ]:
sub=pd.DataFrame(columns=['posting_id','matches'])

In [ ]:
sub['posting_id']=pd.Series(df_test['posting_id'])
sub['matches']=pd.Series(string_list)

In [ ]:
sub.to_csv('submission.csv',index=False)